In [1]:
import sqlite3
import pandas as pd
import numpy as np
import time
from utils import *
from ReactNode import ReactNode
from ChemNode import ChemNode
from MCTS import MCTS
from tree_utils import *

In [2]:
conn = sqlite3.connect('buyable.db')
buyable = conn.cursor()
retrobiocat = pd.read_pickle("data/final_test_retrobiocat.pkl")
analyzer = Retrosim(reference_data_path='data/feasible_rdenzyme_templates.pkl')

In [3]:
smile = "C#C[C@]1([C@H](C[C@@H](O1)N2C=NC3=C(N=C(N=C32)F)N)O)CO"
root = ChemNode(smile, 0, None, buyable, retrobiocat, analyzer, None)

In [4]:
mcts = MCTS(root)

In [8]:
mcts.MCTS(1000)

100%|██████████| 1000/1000 [12:21<00:00,  1.35it/s]


In [9]:
for react in root.reactions:
    print(f'Reaction name: {react.reaction_name}, Score: {react.get_mcts_value()}')
    for precursor in react.precursors:
        print(f'Chem {precursor.smiles}, Score: {precursor.score}, visits: {precursor.visits}, solution: {precursor.solution}')
    print(" ")

Reaction name: 21709, Score: 1.0527605583456805
Chem C#C[C@]1(CSCC[C@H]([NH3+])C(=O)[O-])O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O, Score: 0.95, visits: 8, solution: False
Chem O, Score: -0.7, visits: 7, solution: False
 
Reaction name: PNP, Score: 1.0836921945061269
Chem C#C[C@]1(CO)O[C@@H](OP(=O)([O-])O)C[C@@H]1O, Score: -34.596387499999956, visits: 1333, solution: False
Chem Nc1nc(F)nc2[nH]cnc12, Score: 1.0, visits: 1, solution: True
 
Reaction name: Ketone reduction, Score: 1.054910793068863
Chem C#C[C@]1(CO)O[C@@H](n2cnc3c(N)nc(F)nc32)CC1=O, Score: 0.85975, visits: 16, solution: False
 
Reaction name: Ester hydrolysis (OH), Score: 1.083048499194228
Chem C#C[C@]1(COC(=O)CC)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O, Score: 1.9, visits: 17, solution: False
 
Reaction name: Aldehyde reduction, Score: 1.060551418068863
Chem C#C[C@]1(C=O)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O, Score: 0.95, visits: 16, solution: False
 
Reaction name: Ene hydrolysis, Score: 1.0640484991942278
Chem C#C[C@@]1(CO)C=

In [10]:
root.solution

True

In [11]:
# Prune the tree starting from the root
prune_tree(root)
paths = generate_paths(root)
print(len(paths))

3


In [15]:
path_explorer(paths[1])

BRANCH 0:
-----------------
Chem: C#C[C@]1([C@H](C[C@@H](O1)N2C=NC3=C(N=C(N=C32)F)N)O)CO
Reaction: 27744
SPLIT: Chem1: C#C[C@]1(CO)O[C@H](OP(=O)([O-])[O-])C[C@@H]1O, Chem2: Nc1nc(F)nc2[nH]cnc12

Reaction: 27660
Chem: C#C[C@]1(COP(=O)([O-])[O-])OC(O)C[C@@H]1O
Reaction: 12823
SPLIT: Chem1: C#C[C@](O)(C=O)COP(=O)([O-])[O-], Chem2: CC=O

Reaction: Primary alcohol oxidation
Chem: C#C[C@](O)(CO)COP(=O)([O-])[O-]
Reaction: Phosphorylation
Chem: C#CC(O)(CO)CO
BUYABLE

BRANCH 1:
-----------------
Chem: CC=O
BUYABLE

BRANCH 2:
-----------------
Chem: Nc1nc(F)nc2[nH]cnc12
BUYABLE



In [ ]:
conn2 = sqlite3.connect('databases/Available2.db')
avail = conn2.cursor()

def check_available(smiles, database):
    database.execute('SELECT 1 FROM molecules WHERE SMILES = ?', (smiles,))
    result = database.fetchone()
    return bool(result)

In [31]:
check_available("CC=O", avail)

(1,)


True